# JS03 - Documents Store

Documents in GDN are JSON objects. These objects can be nested (to any depth) and may contain lists. Each document has a unique primary key that identifies it within its collection. Furthermore, each document is uniquely identified by its document handle across all collections. Different revisions of the same document (identified by its handle) can be distinguished by their document revision. Any transaction only ever sees a single revision of a document.

For example:

In [ ]:
{
  "_id" : "myusers/3456789",
  "_key" : "3456789",
  "_rev" : "14253647",
  "firstName" : "John",
  "lastName" : "Doe",
  "address" : {
    "street" : "Road To Nowhere 1",
    "city" : "Gotham"
  },
  "hobbies" : [
    {name: "swimming", howFavorite: 10},
    {name: "biking", howFavorite: 6},
    {name: "programming", howFavorite: 4}
  ]
}

All documents contain special attributes:

- the document handle is stored as a string in `_id`
- the document's primary key in `_key`
- the document revision in `_rev`

The user can specify the value of the `_key` attribute when creating a document. The `_id` and `_key` values are immutable once the document has been created. The `_rev` value is maintained by GDN automatically.

## Pre-requisite

Let's assume:

- You have already made a tenant account, and have a username and password
- You have installed the jsC8 drivers as explained in section 01
- You have generated an API Key as explained in section 01

In [ ]:
#/* run this once to install javascript kernal and jsc8 in google colab, then reload, and then skip this
!npm install jsc8
!npm install -g --unsafe-perm ijavascript
!ijsinstall --install=global  # */

## 1. Connect to GDN

In [ ]:
let emailId, password;
let fed_url = "https://gdn.paas.macrometa.io";

In [ ]:
$$.input(
  { prompt: "Please, type fedration url, leave blank to use default and press enter: " },
  (error, url) => {
      if(url)
        fed_url = url;
    $$.done();
  }
);

In [ ]:
$$.input(
  { prompt: "Please, type your email and press enter: " },
  (error, email) => {
    emailId = email;
    $$.done();
  }
);

In [ ]:
$$.input(
  { prompt: "Please, type your password and press enter: ", password: true },
  (error, pass) => {
    password = pass;
    $$.done();
  }
);

In [ ]:
const jsc8 = require("jsc8");

// ----- simple way  -----
const client = new jsc8({
  url: fed_url,
});

client
  .login(emailId, password)
  .then((result) => console.log("Login successfully", result))
  .catch((err) => console.error("Error while login", err.message));

// ----- with token -----
/* const client = new jsc8({
  url: "https://gdn.paas.macrometa.io",
  token: "XXXX",
  fabricName: "_system",
}); */

// ----- with apikey-----
/* const client = new jsc8({
  url: "https://gdn.paas.macrometa.io",
  apiKey: "XXXX",
  fabricName: "_system",
}); */

## 2. Get GeoFabric Details

To get the details of fabric:

In [ ]:
// you might not need this, but if you wanted to select a
// specific GeoFabric you can find out whats available by executing this code!
const getGeoFabricMetaData = async () => {
  console.log("Get geo fabric details...");

  try {
    const fabrics = await client.get();
    console.log(fabrics);
  } catch (err) {
    console.error("Error while fetching GeoFabric details", err.message);
  }
};

getGeoFabricMetaData();

## 3. Create Collection

We can now create a collection in the fabric. To do this, first, you connect to fabric and then create a collection called employees.

The below example shows the steps:

In [ ]:
const collection_name = "employees";

const createCollection = async () => {
  let collectionDetails;
  try {
    const isCollectionExists = await client.hasCollection(collection_name);
    if (isCollectionExists) {
      console.log("Collection exists with name ", collection_name);
      return;
    }
    collectionDetails = await client.createCollection(collection_name);
    console.log("Collection created! ", collection_name);
  } catch (err) {
    console.error("Collection creation did not succeed due to ", err.message);
  }
};

createCollection();

## 4. Create Index

Let's add a `hash_index` called emails to our collection employees. Please refer to the reference guide for details on other available index types.

In [ ]:
const addHashIndex = async () => {
  console.log("Add Hash Index");
  try {
    const hashIndex = await client.addHashIndex(
      collection_name,
      (fields = ["firstname", "lastname"]),
      { unique: true }
    );
    console.log("Index created: ", hashIndex);
  } catch (err) {
    console.error("Error while adding Hash Index", err.message);
  }
};

addHashIndex();

## 5. Insert Documents

Let's insert documents into the employees collection as shown below.

In [ ]:
let docs = [
  {
    _key: "James",
    firstname: "James",
    lastname: "Kirk",
    email: "email",
  },
  {
    _key: "Han",
    firstname: "Han",
    lastname: "Solo",
    email: "email",
  },
  {
    _key: "Bruce",
    firstname: "Bruce",
    lastname: "Wayne",
    email: "email",
  },
];

const insertDocument = async () => {
  try {
    await client.insertDocument(collection_name, docs);
    console.log("Documents inserted");
  } catch (err) {
    console.error("Error while inserting documents", err.message);
  }
};

insertDocument();

## 6. Query documents using C8QL

C8QL is C8's query language. You can execute C8QL query on our newly created collection employees to get its contents.

The query `FOR employee IN employees RETURN employee` is equivalent to SQL's SELECT query.

In [ ]:
const c8Queries = async () => {
  try {
    let docs = await client.executeQuery(
      "FOR employee IN employees RETURN employee"
    );
    console.log(docs);
  } catch (err) {
    console.error("Error while query documents", err.message);
  }
};

c8Queries();

## 7. Get realtime updates

Example for real-time updates from a collection in fabric:

> Note: You need to enable Stream from the Macrometa Dashboard for this to work. Do this by opening up the dashboard, selecting collections, finding the collection named "employees" and selecting "enable Stream"

Now run the cell below, return to the Macrometa Dashboard and modify and save fields in the collection. You should see the real-time updates below as soon as you hit save.

The WebSocket will time out after 60 seconds if no updates are seen, so you might need to re-run the cell code if you see a timed-out message below.

In [ ]:
const callback_fn = (collection) => {
  console.log("Connection open on ", collection);
};

const realTimeListener = async () => {
  try {
    const listener = await client.onCollectionChange(collection_name);

    listener.on("message", (event) => {
      const encodedMessage = JSON.parse(event).payload;
      let data = {};
      const decodedMessage = atob(encodedMessage);

      if (decodedMessage.length !== 0) {
        data = JSON.parse(decodedMessage);
      }
      console.log("message=>", data);
    });
    listener.on("open", () => {
      callback_fn(collection_name);
    });
    listener.on("close", () => console.log("connection closed"));
  } catch (err) {
    console.error("Error while getting realtime updates", err.message);
  }
};

realTimeListener();

## 8. Time to tidy up!

That was great! Now let's tidy up by removing the collection we created.

In [ ]:
const deleteCollection = async () => {
  try {
    await client.deleteCollection(collection_name);
    console.log("Collection Deleted: ", collection_name);
  } catch (err) {
    console.error("Error while deleting collection ", err.message);
  }
};

deleteCollection();

## Section Completed!

TBC...